In [110]:
import pandas as pd
import vk_api
import requests
import re

In [111]:
access_token=#секрет
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

In [112]:
# нахожу id города
vk.database.getCities(q='белгород')

{'count': 10,
 'items': [{'id': 26, 'title': 'Белгород', 'country': 'Белгородская область'},
  {'id': 3508,
   'title': 'Белгород-Днестровский',
   'area': 'Белгород-Днестровский район',
   'region': 'Одесская область'},
  {'id': 1012699,
   'title': 'Белгород (поселок)',
   'area': 'Навлинский район',
   'region': 'Брянская область'},
  {'id': 1009908,
   'title': 'Белгород-22',
   'area': 'Грайворонский район',
   'region': 'Белгородская область'},
  {'id': 5488173,
   'title': 'Белгород-18',
   'area': 'Корочанский район',
   'region': 'Белгородская область'},
  {'id': 1106020,
   'title': 'Белгородский',
   'area': 'Веневский район',
   'region': 'Тульская область'},
  {'id': 1042569,
   'title': 'Белгородка',
   'area': 'Горшеченский район',
   'region': 'Курская область'},
  {'id': 1700249,
   'title': 'Белгородское',
   'area': 'Сандыктауский район',
   'region': 'Акмолинская область'},
  {'id': 1127492,
   'title': 'Белгородское',
   'area': 'Смидовичский район',
   'region': '

In [113]:
# получаю список пользователей из Белгорода
json_users = vk.users.search(city=26, fields = 'sex,city,bdate', count=1000)['items']

In [115]:
# привожу в нужный формат city
for string in json_users:
    try:
        string['city'] = string['city']['title']
    except:
        continue

In [116]:
# функция для выбора года рождения
def get_year(date):
    year = re.findall('[0-9]{4}', date)
    if len(year):
        return year[0]
    return '0'

In [117]:
# заменяю пропуски, выбираю нужные столбцы и меняю значения некоторых столбцов
data = pd.DataFrame(json_users)[['first_name', 'last_name', 'city', 'sex', 'bdate']]
data['city'] = data['city'].fillna('Белгород')
data['bdate'] = data['bdate'].fillna('1')
data['bdate'] = data['bdate'].apply(lambda x: get_year(x))
data['sex'] = data['sex'].apply(lambda x: 'женщина' if x == 1 else 'мужчина')

In [118]:
data

,first_name,last_name,city,sex,bdate
0,Евгений,Романенко,Белгород,мужчина,1998
1,Данил,Золотарев,Белгород,мужчина,2002
2,Ivan,Popaz,Белгород,мужчина,2004
3,Давид,Хабиямбере,Белгород,мужчина,2000
4,Dmitry,Zhuravlev,Белгород,мужчина,2003
...,...,...,...,...,...
733,Виктор,Саулов,Белгород,мужчина,0
734,Юрий,Агарков,Белгород,мужчина,1949
735,Виталий,Данилин,Белгород,мужчина,1978
736,Ярослав,Воробьев,Белгород,мужчина,1985


In [119]:
# сохраняю датасет, где имеются пользователи с годом рождения
data[data['bdate'] != '0'].to_csv('hw.csv')